In [ ]:
# Ensure GPU is being used
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from transformers import AutoModelForImageClassification, AutoImageProcessor
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Required input size for ViT
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load CIFAR10
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [ ]:
def split_dataset(dataset, split_ratios):
    train_size = int(split_ratios[0] * len(dataset))
    val_size = int(split_ratios[1] * len(dataset))
    test_size = len(dataset) - train_size - val_size
    return random_split(dataset, [train_size, val_size, test_size])

split_ratios_list = [
    [0.6, 0.1, 0.3],
    [0.6, 0.3, 0.1],
    [0.6, 0.2, 0.2],
    [0.5, 0.1, 0.4],
    [0.5, 0.4, 0.1],
    [0.5, 0.25, 0.25]
]

In [ ]:
model_name = 'google/vit-base-patch16-224-in21k'
model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=10).to(device)
processor = AutoImageProcessor.from_pretrained(model_name)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
def train_and_evaluate(train_loader, val_loader, test_loader, model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Evaluate the model
    model.eval()
    def evaluate(loader):
        y_true, y_pred = [], []
        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images).logits
                _, predicted = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())
        return accuracy_score(y_true, y_pred)

    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    test_acc = evaluate(test_loader)
    return train_acc, val_acc, test_acc

In [ ]:
results = []
for split_ratios in split_ratios_list:
    print(f"\nRunning for split ratios: Train-Val-Test = {split_ratios}")
    train_set, val_set, test_set = split_dataset(train_data, split_ratios)

    # DataLoader
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    train_acc, val_acc, test_acc = train_and_evaluate(train_loader, val_loader, test_loader, model, criterion, optimizer)
    results.append({
        "split_ratios": split_ratios,
        "train_accuracy": train_acc,
        "val_accuracy": val_acc,
        "test_accuracy": test_acc
    })
    print(f"Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}")

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
print("\nSummary of Results:")
print(df)